In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf=SparkConf().setAppName("BDProject").setMaster("local[3]")

In [4]:
sc=SparkContext(conf=conf)

In [5]:
rawBadges=sc.textFile("travel.stackexchange.com\Badges.xml")

In [6]:
rawUsers=sc.textFile("travel.stackexchange.com\\Users.xml")

In [19]:
rawUsers.take(3)

['<?xml version="1.0" encoding="utf-8"?>',
 '<users>',
 '  <row Id="-1" Reputation="1" CreationDate="2011-06-21T15:16:44.253" DisplayName="Community" LastAccessDate="2011-06-21T15:16:44.253" WebsiteUrl="http://meta.stackexchange.com/" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I\'m not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I\'m a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="0" UpV

In [24]:
def cleaningRDD(RDDIn):
    def cleaning(listIn):
        listOut=[]
        for x in listIn:
            if listIn.index(x)%2==0:
                listOut.append(x.replace(' ','').replace('<','').replace('=',''))
            else:
                listOut.append(x)
        return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
    return RDDIn.filter(lambda x: x.startswith('  <row Id=')).\
map(lambda x: x.split('"')).map(cleaning)

In [25]:
badges=cleaningRDD(rawBadges)

In [26]:
badges.take(1)

[{'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '2',
  'rowId': '1'}]

In [22]:
users=cleaningRDD(rawUsers)

In [27]:
users.take(1)

[{'AboutMe': "&lt;p&gt;Hi, I'm not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I'm a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;",
  'AccountId': '-1',
  'CreationDate': '2011-06-21T15:16:44.253',
  'DisplayName': 'Community',
  'DownVotes': '14237',
  'LastAccessDate': '2011-06-21T15:16:44.253',
  'Location': 'on the server farm',
  'Reputation': '1',
  'UpVotes': '3147',
  'Views': '0',
  'WebsiteUrl